In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score, confusion_matrix 
import matplotlib.pyplot as plt

In [3]:
credit_card = pd.read_csv('creditcard.csv')
X = credit_card.drop(columns='Class', axis=1)
X['logAmount'] =  np.log(X['Amount'] + 1.0)
X = X.drop(columns=['Amount', 'Time'], axis=1)
y = credit_card.Class.values
np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X, y)
# y_train, y_test = np.array([-1 if y==0 else 1 for y in y_train]), np.array([-1 if y==0 else 1 for y in y_test])
y_train, y_test = y_train.reshape(-1,1), y_test.reshape(-1,1)
y_train, y_test = y_train.astype(np.float32), y_test.astype(np.float32)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train).astype(np.float32)
X_test = scaler.transform(X_test).astype(np.float32)

In [4]:
class LogModel():
    def __init__(self, optimizer=None, learning_rate=0.01):
        super(LogModel, self).__init__()
        if optimizer:
            self.optimizer = optimizer(learning_rate)
        else:
            self.optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, decay=0.01)

    def loss(self, pred, true):
        loss = tf.keras.losses.BinaryCrossentropy()
        return loss(pred, true)

    def accuracy(self, pred, true):
        check_prediction = tf.equal(tf.round(pred), true)
        return tf.reduce_mean(tf.cast(check_prediction, tf.float32)) * 100

    def predict(self, x):
        return tf.nn.sigmoid(tf.matmul(x, self.w) + self.b)
        
    def fit(self, x, y, n_epochs, batch_size, validation_data=None):
        self.w = tf.Variable(tf.random.truncated_normal([x.shape[1], 1]), name="weight", trainable=True)
        self.b = tf.Variable(tf.zeros([1]), name='bias', trainable=True)     

        train_data=tf.data.Dataset.from_tensor_slices((x, y))
        train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)
        self.train_loss = []
        self.train_accuracy= []
        if validation_data:
            self.valid_accuracy = []
            self.valid_loss = []
        for step, (batch_x, batch_y) in enumerate(train_data.take(n_epochs), 1):
            # Run the optimization to update W and b values.
            with tf.GradientTape() as g:
                pred = tf.nn.sigmoid(tf.matmul(batch_x, self.w) + self.b)
                loss = self.loss(pred, batch_y)
                self.train_loss.append(loss)
                self.train_accuracy.append(self.accuracy(pred, batch_y))
            # Compute gradients.
            gradients = g.gradient(loss, [self.w, self.b])
            self.optimizer.apply_gradients(zip(gradients, [self.w, self.b]))
            if validation_data:
                x_val, y_val = validation_data
                pred_val = self.predict(x_val)
                self.valid_loss.append(self.loss(pred_val, y_val))
                self.valid_accuracy.append(self.accuracy(pred_val, y_val))
                
                if step % 50 == 0:
                    print("Iteration: %i, loss: %.2f, accuracy: %.2f, loss_val: %.2f, accuracy_val: %.2f" % (step, 
                    self.train_loss[step-1], self.train_accuracy[step-1], self.valid_loss[step-1], self.valid_accuracy[step-1]))
            else:
                if step % 50 == 0:
                    print("Iteration: %i, loss: %.2f, accuracy: %.2f" % (step, self.train_loss[step-1], self.train_accuracy[step-1]))

            


In [5]:
lr = LogModel(optimizer=tf.keras.optimizers.Adam, learning_rate=0.03)
lr.fit(X_train, y_train, n_epochs=5000, batch_size=500)

Iteration: 50, loss: 3.23, accuracy: 82.80
Iteration: 100, loss: 1.23, accuracy: 95.60
Iteration: 150, loss: 0.63, accuracy: 97.60
Iteration: 200, loss: 0.71, accuracy: 96.40
Iteration: 250, loss: 0.55, accuracy: 97.20
Iteration: 300, loss: 0.45, accuracy: 97.60
Iteration: 350, loss: 0.59, accuracy: 96.60
Iteration: 400, loss: 0.58, accuracy: 96.40
Iteration: 450, loss: 0.44, accuracy: 97.60
Iteration: 500, loss: 0.23, accuracy: 99.00
Iteration: 550, loss: 0.11, accuracy: 99.60
Iteration: 600, loss: 0.04, accuracy: 100.00
Iteration: 650, loss: 0.07, accuracy: 99.80
Iteration: 700, loss: 0.03, accuracy: 100.00
Iteration: 750, loss: 0.03, accuracy: 100.00
Iteration: 800, loss: 0.05, accuracy: 99.80
Iteration: 850, loss: 0.05, accuracy: 99.80
Iteration: 900, loss: 0.05, accuracy: 99.80
Iteration: 950, loss: 0.02, accuracy: 100.00
Iteration: 1000, loss: 0.02, accuracy: 100.00
Iteration: 1050, loss: 0.05, accuracy: 99.80
Iteration: 1100, loss: 0.04, accuracy: 99.80
Iteration: 1150, loss: 0.

In [6]:
y_test_hat = np.round(lr.predict(X_test))
# y_test_hat_probs = lr.predict_proba(X_test)[:,1]
test_accuracy = accuracy_score(y_test, y_test_hat)*100
# test_auc_roc = roc_auc_score(y_test, y_test_hat_probs)*100
print('Confusion matrix:\n', confusion_matrix(y_test, y_test_hat))
print('Test accuracy: %.4f %%' % test_accuracy)
# print('Test AUC: %.4f %%' % test_auc_roc)

Confusion matrix:
 [[71059    30]
 [   22    91]]
Test accuracy: 99.9270 %


In [7]:
print(classification_report(y_test, y_test_hat, digits=6))

              precision    recall  f1-score   support

         0.0   0.999690  0.999578  0.999634     71089
         1.0   0.752066  0.805310  0.777778       113

    accuracy                       0.999270     71202
   macro avg   0.875878  0.902444  0.888706     71202
weighted avg   0.999298  0.999270  0.999282     71202

